<a href="https://colab.research.google.com/github/ankitg-02/extern/blob/main/Webacy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data=pd.read_excel("/content/compiled_risk_data (1).xlsx")
Webacy=pd.DataFrame(data)
Webacy.sample(5)

,project_name,Smart contract address,Blog post link,Audit website,Chain,Is_closed_source,hidden_owner,anti_whale_modifiable,Is_anti_whale,Is_honeypot,...,centralized_risk_high,centralized_risk_low,event_setter,external_dependencies,immutable_states,reentrancy_without_eth_transfer,incorrect_inheritance_order,shadowing_local,events_maths,Summary/rationale of risk tags marked true
795,Data Analytics,323b743e4bb60dd4ca8f4d6a18b5f7589354c6ab,https://wp.hacken.io/wp-content/uploads/2022/0...,https://hacken.io/,Ethereum,False,False,False,False,False,...,0.0,0.0,False,False,False,False,False,False,False,The smart contract doesn't follow the coding s...
399,Data Analytics,80219e458290ff663dd4adc4976019495f09aa56,https://ipfs.io/ipfs/QmWQvTdiD3fSwJgasPLppHZKP...,https://kudelskisecurity.com,Arbitrum,False,False,False,False,False,...,0.0,0.0,False,True,True,False,False,False,False,he primary issue is that the parseDuration fun...
682,Data Analytics,0x250e76987d838a75310c34bf422ea9f1AC4Cc906,https://rekt.news/polynetwork-rekt/,https://rekt.news/,Ethereum,False,False,False,False,False,...,0.0,0.0,False,False,False,False,False,False,False,NaN
528,Data Analytics,0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E,https://www.immunebytes.com/blog/conic-finance...,https://www.immunebytes.com/,Ethereum,False,False,False,False,False,...,0.0,0.0,False,False,False,True,False,False,False,NaN
722,Data Analytics,0x6Bb61215298F296C55b19Ad842D3Df69021DA2ef,https://rekt.news/furucombo-rekt/,https://rekt.news/,Ethereum,False,False,False,False,False,...,0.0,0.0,False,True,False,False,False,False,False,NaN


In [4]:
Webacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 38 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   project_name                                967 non-null    object 
 1   Smart contract address                      967 non-null    object 
 2   Blog post link                              967 non-null    object 
 3   Audit website                               967 non-null    object 
 4   Chain                                       967 non-null    object 
 5   Is_closed_source                            967 non-null    bool   
 6   hidden_owner                                967 non-null    bool   
 7   anti_whale_modifiable                       967 non-null    bool   
 8   Is_anti_whale                               967 non-null    bool   
 9   Is_honeypot                                 967 non-null    bool   
 10  buy_tax       

In [9]:
for i in Webacy.columns[4:36]:
  print(Webacy[i].value_counts())

Chain
Ethereum    849
Arbitrum     50
Polygon      40
Optimism     28
Name: count, dtype: int64
Is_closed_source
False    821
True     146
Name: count, dtype: int64
hidden_owner
False    817
True     150
Name: count, dtype: int64
anti_whale_modifiable
False    855
True     112
Name: count, dtype: int64
Is_anti_whale
False    829
True     138
Name: count, dtype: int64
Is_honeypot
False    878
True      89
Name: count, dtype: int64
buy_tax
False    850
True     117
Name: count, dtype: int64
sell_tax
False    848
True     119
Name: count, dtype: int64
slippage_modifiable
False    828
True     139
Name: count, dtype: int64
Is_blacklisted
False    891
True      76
Name: count, dtype: int64
can_take_back_ownership
False    786
True     181
Name: count, dtype: int64
owner_change_balance
False    763
True     204
Name: count, dtype: int64
is_airdrop_scam
False    902
True      65
Name: count, dtype: int64
selfdestruct
False    856
True     111
Name: count, dtype: int64
trust_list
False    827


In [14]:
risk_columns=[]
for i in Webacy.columns[4:36]:
  risk_columns.append(i)
print(risk_columns)

['Chain', 'Is_closed_source', 'hidden_owner', 'anti_whale_modifiable', 'Is_anti_whale', 'Is_honeypot', 'buy_tax', 'sell_tax', 'slippage_modifiable', 'Is_blacklisted', 'can_take_back_ownership', 'owner_change_balance', 'is_airdrop_scam', 'selfdestruct', 'trust_list', 'is_whitelisted', 'is_fake_token', 'illegal_unicode', 'exploitation', 'bad_contract', 'reusing_state_variable', 'encode_packed_collision', 'encode_packed_parameters', 'centralized_risk_medium', 'centralized_risk_high', 'centralized_risk_low', 'event_setter', 'external_dependencies', 'immutable_states', 'reentrancy_without_eth_transfer', 'incorrect_inheritance_order', 'shadowing_local']
